In [ ]:
from IPython.display import display, HTML
from sympy.abc import alpha
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
# Brightway imports
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import brightway2 as bw

In [ ]:
import pandas as pd
import numpy as np
import datetime
import os

In [ ]:
# Custom functions
from lca_calculation_functions import get_inventory_dataset, init_simple_lca, multi_lcia, multi_contribution_analysis, direct_technosphere_contribution_multi_activities_fixed,  calculate_projected_impacts
from visualisation_functions import plot_multilca_impacts, plot_contribution_analysis, plot_production_impacts, plot_incremental_impacts, plot_iwplus_contributions, plot_scenario_production_comparison, pie_charts_technosphere_contribution

# Set projects, LCI and LCIA methods

In [ ]:
BW_PROJECT = 'regioinvent_2025' # insert your project name here
bd.projects.set_current(BW_PROJECT)
bd.databases

In [ ]:
EI_DB = 'ecoinvent-3.10-cutoff'
RI_DB = 'Regioinvent'
LIB_RM_DB = 'LIB raw materials'

In [ ]:
for act in bw.Database("LIB raw materials"):
    print(act)

In [ ]:
bd.databases['LIB raw materials']


## Import LCI

In [ ]:
# (mineral name: activity name, reference product, location)
INVENTORIES = {
    # From Ecoinvent 3.10
    "Copper concentrate CA": ('copper mine operation and beneficiation, sulfide ore', 
                           'copper concentrate, sulfide ore', 
                           'CA'),
    "Copper concentrate CA (gold)": ('gold-silver mine operation and beneficiation', 
                                  'copper concentrate, sulfide ore', 
                                  'CA-QC'),
    "Molybdenite CA": ('copper mine operation and beneficiation, sulfide ore', 
                    'molybdenite', 
                    'CA'),
    "Sulfidic tailings copper CA": ('treatment of sulfidic tailings, from copper mine operation, tailings impoundment',
                                 'sulfidic tailings, from copper mine operation', 
                                 'CA'),
    "Nickel concentrate CA": ('nickel mine operation and benefication to nickel concentrate, 16% Ni',
                           'nickel concentrate, 16% Ni',
                           'CA-QC'),
    "Sulfidic tailings nickel CA": ('treatment of sulfidic tailings, from nickel mine operation, tailings impoundment',
                           'sulfidic tailings, from nickel mine operation',
                           'CA'),
    
    # From Istrate et al (2024)
    #"Nickel ore mining": ('nickel ore mining, average excluding China',
    #                      'nickel ore, mined',
    #                      'GLO'),
    #"Nickel concentrate": ('nickel concentration, average excluding China',
    #                       'nickel concentrate',
    #                       'GLO'),
    #"Nickel matte": ('nickel matte production, nickel-cobalt sulphite and nickel sub materials, mass allocation',
    #                 'nickel matte',
    #                 'GLO') # relevant for smelting? intermediate product obtained from smelting nickel concentrate
}

In [ ]:
# # (mineral name: activity name, reference product, location)
# INVENTORIES = {
#     ## From EI
#     "Neodymium":        ("rare earth oxides production, from rare earth carbonate concentrate", "neodymium oxide", "RoW"),
#     "Copper concentrate, Canada": ('copper mine operation and beneficiation, sulfide ore', 'copper concentrate, sulfide ore', 'CA'),
#     "Market for copper, cathode": ('market for copper, cathode', 'copper, cathode', 'GLO'),
#     
#     # From Istrate et al (2024)
#     "Lithium hydroxide, brine":        ("lithium hydroxide production, Salar de Atacama", "lithium hydroxide, battery grade", "CL"),
#     "Lithium hydroxide, spodumene":    ("lithium hydroxide production, from Australian spodumene", "lithium hydroxide, battery grade", "CN"),
#     "Cobalt":                ("cobalt sulfate production, from copper-cobalt ore, economic allocation", "cobalt sulfate", "CN"),
#     "Nickel":                ("nickel sulfate production, average excluding China, economic allocation", "nickel sulfate", "GLO"),
#     "Graphite, natural":     ("natural graphite production, battery grade, from Heilongjiang", "natural graphite, battery grade", "CN"),
#     "Graphite, synthetic":   ("graphite powder coating", "synthetic graphite, battery grade", "CN"),
#     
#     "Lithium carbonate, brine":        ("lithium carbonate production, Salar de Atacama", "lithium carbonate, battery grade", "CL"),
#     "Lithium carbonate, spodumene":    ("lithium carbonate production, from Australian spodumene", "lithium carbonate, battery grade", "CN"),
# }

In [ ]:
INVENTORIES_EI = {
    "Neodymium": ("market for neodymium oxide", "neodymium oxide", "GLO"), # CAN not here
    "Copper": ("market for copper, cathode", "copper, cathode", 'GLO'),    
    "Lithium carbonate": ("market for lithium carbonate", "lithium carbonate", "GLO"),
    "Cobalt oxide": ("market for cobalt oxide", "cobalt oxide", "GLO"),
    "Nickel": ("market for nickel, class 1", "nickel, class 1", "GLO"),
    "Graphite": ("market for graphite", "graphite", "GLO")
}

In [ ]:
# If we take consumption market activities 
# (mineral name: activity name, reference product, location)
INVENTORIES_RI = {
    "Neodymium": ("consumption market for neodymium oxide", "neodymium oxide", "US"), # CAN not here
    "Copper": ("consumption market for copper, cathode", "copper, cathode", 'CA'),    
    "Lithium carbonate": ("consumption market for lithium carbonate", "lithium carbonate", "CA"),
    "Cobalt oxide": ("consumption market for cobalt oxide", "cobalt oxide", "CA"),
    "Nickel": ("consumption market for nickel, class 1", "nickel, class 1", "CA"),
    "Graphite": ("consumption market for graphite", "graphite", "US"), # US     
}

In [ ]:
INVENTORIES_EI_ds = get_inventory_dataset(INVENTORIES_EI, database_names=[EI_DB, LIB_RM_DB])

In [ ]:
INVENTORIES_RI_ds = get_inventory_dataset(INVENTORIES_RI, database_names=[RI_DB])

In [ ]:
INVENTORIES_ds = get_inventory_dataset(INVENTORIES, database_names=[EI_DB, LIB_RM_DB])

## Pick LCIA methods

In [ ]:
# We can also import some from IW+2.1
expert_ei310 = 'data/IW+2.1/impact_world_plus_21_brightway2_expert_version_ei310.5535d12bedce3770ffef004e84229fd1.bw2package'
bw.BW2Package.import_file(expert_ei310)

In [ ]:
# Filter and display metho|ds that contain "IMPACT World+" in their names
iw_methods = [method for method in bd.methods if "impact world+" in " ".join(method).lower()]
df_iw_methods = pd.DataFrame(iw_methods, columns=["Method", "Impact Category", "Subcategory"])
df_iw_methods.to_csv(r'data/IW+2.1/iw_methods_3.10.csv', index=False)

In [ ]:
# To take them all
IMPACT_METHODS_ALL = {method[-1]: method for method in iw_methods}
IMPACT_METHODS_ALL

In [ ]:
IMPACT_METHODS = {
#'Climate change ST': ('IMPACT World+ Midpoint 2.0.1_regionalized','Midpoint', 'Climate change, short term'), 
'Total HH': ('IMPACT World+ Damage 2.1 for ecoinvent v3.10','Human health', 'Total human health'), 
'Total EQ': ('IMPACT World+ Damage 2.1 for ecoinvent v3.10','Ecosystem quality', 'Total ecosystem quality'), 
}


# Calculate specific impacts (e.g. per mass)

## With inventories from EI and other LCI

In [ ]:
# Initialize LCA object
lca = init_simple_lca(INVENTORIES_ds["Copper concentrate CA"])

In [ ]:
specific_lca = {}
for rm in INVENTORIES_ds:
    impacts = multi_lcia(lca, INVENTORIES_ds[rm], IMPACT_METHODS)
    specific_lca[rm] = impacts

specific_lca = pd.DataFrame(specific_lca).T
specific_lca = specific_lca.reset_index().rename(columns={
    'index': 'Commodity',
})

In [ ]:
specific_lca

## With Regioinvent

In [ ]:
# Initialize LCA object
lca_ri = init_simple_lca(INVENTORIES_RI_ds["Neodymium"])

In [ ]:
specific_lca_ri = {}
for rm in INVENTORIES_RI_ds:
    impacts = multi_lcia(lca_ri, INVENTORIES_RI_ds[rm], IMPACT_METHODS)
    specific_lca_ri[rm] = impacts

specific_lca_ri = pd.DataFrame(specific_lca_ri).T
specific_lca_ri = specific_lca_ri.reset_index().rename(columns={
    'index': 'Commodity',
})

In [ ]:
specific_lca_ri

## Plotting

In [ ]:
# Call the function
plot_multilca_impacts(specific_lca, 
                 colors=["#ff7f0e", "#2ca02c"], 
                 save_path="results/specific_lca_results/lca_impacts_ei.png")

In [ ]:
# Call the function
plot_multilca_impacts(specific_lca_ri, 
                 colors=["#ff7f0e", "#2ca02c"], 
                 save_path="results/demand_lca_results/specific_lca_impacts_regioinvent.png")

In [ ]:
#specific_lca.to_csv(f"results/specific_lca_results/specific_results_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index_label="Raw material")

## Contribution analysis

In [ ]:
# Initialize a dictionary to store contribution results for each inventory
contribution_results = {}

# Loop through each inventory and perform contribution analysis
for rm_name, rm_ds in INVENTORIES_ds.items():
    lca = init_simple_lca(rm_ds)
    contributions = multi_contribution_analysis(lca, IMPACT_METHODS, top_n=10, threshold=0.01)  # Set your threshold here
    contribution_results[rm_name] = contributions

# Convert the results into a more readable format for analysis
contribution_dfs = {}

for rm_name, impacts in contribution_results.items():
    for impact_name, contributions in impacts.items():
        df = pd.DataFrame(contributions)
        df["Inventory"] = rm_name
        df["Impact Category"] = impact_name
        contribution_dfs[(rm_name, impact_name)] = df

# Combine all individual DataFrames into one for easy viewing
contribution_analysis_df = pd.concat(contribution_dfs.values(), ignore_index=True)

# Set "Inventory" and "Impact Category" as the row indices
contribution_analysis_df.set_index(["Inventory", "Impact Category"], inplace=True)


In [ ]:
contribution_analysis_df

In [ ]:
contribution_analysis_df.to_excel(r'results/link_with_metallican/contribution_analysis_to_ep.xlsx')

In [ ]:
# Example usage
inventory_names = ["Copper concentrate CA",
                   "Copper concentrate CA (gold)",
                   "Molybdenite CA", 
                   "Sulfidic tailings copper CA",
                   "Sulfidic tailings nickel CA", 
                   ]
plot_contribution_analysis(contribution_analysis_df, inventory_names, 
                           save_path="results/specific_lca_results/contribution_analysis")

In [ ]:
#contribution_analysis_df.to_csv(f"results/specific_lca_results/contribution_analysis/contribution_analysis_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv")

In [ ]:
tech_contribution_df = direct_technosphere_contribution_multi_activities_fixed(lca, activities=INVENTORIES_ds, lcia_methods=IMPACT_METHODS, amount=1)

In [ ]:
tech_contribution_df

In [ ]:
tech_contribution_df.to_csv(r'results/link_with_metallican/tech_contribution.csv', index=False)

In [ ]:
import plotly.express as px

In [ ]:
def pie_charts_technosphere_contribution(df, activity_col='Activity', method_col='LCIA Method',
                                           flow_name_col='Flow Name', location_col='Flow Location',
                                           value_col='Absolute Contribution', legend_size=10,
                                           percentage_threshold=5, save_path=None):
    """
    Generate interactive pie charts for each combination of activity and LCIA method.
    Contributions < percentage_threshold% are aggregated into an 'Other' category.

    Parameters:
    - df (pd.DataFrame): Input DataFrame with required columns.
    - activity_col (str): Column name for Activity (e.g., minerals).
    - method_col (str): Column name for LCIA Method.
    - flow_name_col (str): Column name for Flow Name.
    - location_col (str): Column name for Flow Location.
    - value_col (str): Column name for contributions (e.g., Absolute Contribution).
    - legend_size (int): Font size for the legend.
    - percentage_threshold (float): Threshold percentage for aggregating into 'Other'.
    - save_path (str): Path to save interactive HTML graphs (optional).
    """
    import plotly.express as px

    # Merge 'Flow Name' and 'Flow Location' into a single label
    df['Flow Label'] = df[flow_name_col] + ' (' + df[location_col] + ')'

    # Get unique Activities and LCIA Methods
    unique_activities = df[activity_col].unique()
    unique_methods = df[method_col].unique()

    # Loop through each activity and LCIA Method
    for activity in unique_activities:
        for method in unique_methods:
            subset = df[(df[activity_col] == activity) & (df[method_col] == method)]

            if subset.empty:
                continue

            # Calculate total contribution and percentages
            total_contribution = subset[value_col].sum()
            subset['Percentage'] = (subset[value_col] / total_contribution) * 100

            # Aggregate small contributions into 'Other'
            above_threshold = subset[subset['Percentage'] >= percentage_threshold]
            below_threshold = subset[subset['Percentage'] < percentage_threshold]

            if not below_threshold.empty:
                other_sum = below_threshold[value_col].sum()
                other_row = pd.DataFrame({
                    'Flow Label': ['Other'],
                    value_col: [other_sum],
                    'Percentage': [(other_sum / total_contribution) * 100]
                })
                subset_cleaned = pd.concat([above_threshold, other_row], ignore_index=True)
            else:
                subset_cleaned = above_threshold

            # Create interactive pie chart
            fig = px.pie(
                subset_cleaned,
                names='Flow Label',
                values=value_col,
                title=f'{activity} - {method}',
            )

            # Update legend size
            fig.update_layout(
                legend=dict(font=dict(size=legend_size)),
                title=dict(font=dict(size=14)),
            )

            # Show plot
            fig.show()

            # Save as HTML if a path is provided
            if save_path:
                filename = f"{save_path}/{activity}_{method}_pie_chart.html".replace(' ', '_')
                fig.write_html(filename)
                print(f"Saved: {filename}")

In [ ]:
pie_charts_technosphere_contribution(tech_contribution_df, legend_size=12, percentage_threshold=5, save_path='results/link_with_metallican')

In [ ]:
# Filter to remove midpoint indicators and footprint entries
IMPACT_METHODS_DAMAGE = {
    key: value for key, value in IMPACT_METHODS_ALL.items()
    if value[1] != "Midpoint" and value[0] == "IMPACT World+ Damage 2.1 for ecoinvent v3.10"
}

In [ ]:
IMPACT_METHODS_DAMAGE

In [ ]:
specific_lca_damage = {}
for rm in INVENTORIES_ds:
    impacts = multi_lcia(lca, INVENTORIES_ds[rm], IMPACT_METHODS_DAMAGE)
    specific_lca_damage[rm] = impacts

specific_lca_damage = pd.DataFrame(specific_lca_damage).T
specific_lca_damage = specific_lca_damage.reset_index().rename(columns={
    'index': 'Commodity'
})

In [ ]:
specific_lca_damage

In [ ]:
specific_lca_damage.to_csv(f"results/link_with_metallican/specific_results_hh_eq.csv", index_label="Raw material")

In [ ]:
plot_iwplus_contributions(specific_lca_damage, save_path_eco="results/link_with_metallican/ecosystem_quality_contributions.png", save_path_hh="results/link_with_metallican/human_health_contributions.png")

# Calculate production-related impacts (e.g. scaled with scenarios)

## Import scenarios

They are from the Canadian Climate Institute and can be found [here](https://440megatonnes.ca/insight/canada-critical-minerals-clean-energy-transition/)

In [ ]:
production_existing = pd.read_excel(r'data/Scenarios/scenarios_canadian_climate_institute.xlsx', sheet_name='Production_existing')
production_potential = pd.read_excel(r'data/Scenarios/scenarios_canadian_climate_institute.xlsx', sheet_name='Production_potential')
production_existing = production_existing[production_existing['Scenario']=='Domestic demand scenario']
production_potential = production_potential[production_potential['Scenario']=='Domestic demand scenario']

In [ ]:
production_existing.rename(columns={'Mineral': 'Commodity'}, inplace=True)
production_potential.rename(columns={'Mineral': 'Commodity'}, inplace=True)

In [ ]:
plot_scenario_production_comparison(production_existing, production_potential, save_path='results/demand_lca_results/cci_production_scenarios.png')

## Choose which LCI to associate with each mineral 

In [ ]:
# Define a manual mapping to link the mineral in the scenario with the LCI of our choice
mineral_to_material_ei = {
    "Cobalt": "Cobalt oxide",
    "Copper": "Copper",
    "Lithium": "Lithium carbonate", 
    "Nickel": "Nickel",
    "Graphite": "Graphite",
    "Neodymium": "Neodymium"  
}


In [ ]:
# Define a manual mapping to link the mineral in the scenario with the LCI of our choice
mineral_to_material_ri = {
    "Cobalt": "Cobalt oxide",
    "Copper": "Copper",
    "Lithium": "Lithium carbonate", 
    "Nickel": "Nickel",
    "Graphite": "Graphite",
    "Neodymium": "Neodymium"  
}

## Calculate the production-related impacts of scenarios

In [ ]:
# With EI
projected_impacts_existing_production_ei = calculate_projected_impacts(production_existing, specific_lca_ei, mineral_to_material_ei)
projected_impacts_potential_production_ei = calculate_projected_impacts(production_potential, specific_lca_ei, mineral_to_material_ei)


In [ ]:
# With Regioinvent
projected_impacts_existing_production_ri = calculate_projected_impacts(production_existing, specific_lca_ri,
                                                                       mineral_to_material_ri)
projected_impacts_potential_production_ri = calculate_projected_impacts(production_potential, specific_lca_ri,
                                                                        mineral_to_material_ri)


In [ ]:
projected_impacts_existing_production_ri

In [ ]:
impact_categories = [
#'Climate change ST (kg CO2 eq)',
'Total HH (DALY)',
'Total EQ (PDF.m2.yr)'    
]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Plot impacts per production scenario
plot_production_impacts(projected_impacts_existing_production_ei, production_existing, 
                             impact_categories, 
                             scenario_name='existing_production',
                            lci_used='EI')
plot_production_impacts(projected_impacts_potential_production_ei, production_potential, 
                             impact_categories, scenario_name='potential_production',
                        lci_used='EI')


In [ ]:
# Plot impacts per production scenario
plot_production_impacts(projected_impacts_existing_production_ri, production_existing, 
                             impact_categories, 
                             scenario_name='existing_production',
                             lci_used='Regioinvent')
plot_production_impacts(projected_impacts_potential_production_ri, production_potential, 
                             impact_categories, 
                        scenario_name='potential_production',
                        lci_used='Regioinvent')